## Think Stats - Chapter 1: Exploratory Data Analysis
[link](https://greenteapress.com/wp/think-stats-2e/); 
by Allen B. Downey, Green Tea Press

In [105]:
from __future__ import print_function, division
import nsfg
import thinkstats2
import numpy as np

### Do first babies tend to arrive late?
Questions like this often lead to anecdotes, which suffer from
- **small sample size**
- **selection bias** - people who join the discussion or respond to a survey may only join because they have an extreme opinion
- **confirmation bias** - people who believe something may be more likely to offer examples that confirm it, people who doubt it are more likely to provide examples that disagree
- **inaccuracy**

### 1.1: A statistical approach
These limitations can be remedied using tools like
- **Data collection**
- **Descriptive statistics** - summarize and visualize data
- **Exploratory data analysis (EDA)** - look for patterns, inconsitencies, limitations
- **Estimation** - use sample data to predict population characteristics
- **Hypothesis testing** - where apparent effects are observed, might they have happened by chance?

### 1.2: The National Survey of Family Growth
Has data we can use to see whether first babies tend to come late. <br>
It's a `cross-sectional` study, meaning it captures a snapshot of a group at one point in time (as opposed to a `longitudinal study` which observes the same group repeatedly over time. <br><br>
Ppl who fill out a survey are `respondents` and comprise a `sample` of the `population`. Cross-sectional studies are usually meant to be `representative`, meaning every member of the target population has an equal chance of participating. This survey, however, is `oversampled` toward Hispanics, African-Americans, and teenagers to make sure each group has enough data to draw meaningful inferences. <br><br>
A downside of oversampling is that it's harder to draw conclusions about the general population. <br><br>
The `codebook` that accompanies data gives details about the study design, questions, and structure of the data. <br><br>
[codebook](http://www.cdc.gov/nchs/nsfg/nsfg_cycle6.htm) for this survey

### 1.3: Importing the data
`2002FemPreg.dat.gz` is a gzip-compressed data file in plain text (ASCII) with fixed width columns. Each line is a record. The first row looks like:
>1 1     6 1     11093 1084     9 039 9   0  1 813             1093 13837                        1                5                                                                        116610931166 9201093                111             3    1   12  11         5391 110933316108432411211     2995 1212 69544441116122222 2 2224693215    000000000000000000000000000000000000003410.38939935294273869.3496019830486 6448.2711117047512 91231

`2002FemPreg.dct` is a Stata dictionary file. **Stata** is a statistical software system, and a dictionary is a list of variable names, types, and indices that say where in each line to find that variable. The file looks like:
>infile dictionary {
    _column(1) str12 caseid  %12s  "RESPONDENT ID NUMBER"
   }

This means that the first column is a 12-character string

These two files can be combined to create a `dataframe` like so:

In [106]:
def ReadFemPreg(dct_file='2002FemPreg.dct',
                dat_file='2002FemPreg.dat.gz'):
    dct = thinkstats2.ReadStataDct(dct_file)
    df = dct.ReadFixedWidth(dat_file, compression='gzip')
    CleanFemPreg(df)
    return df

def CleanFemPreg(df):
    df.agepreg /= 100.0

    na_vals = [97, 98, 99]
    df.birthwgt_lb.replace(na_vals, np.nan, inplace=True)
    df.birthwgt_oz.replace(na_vals, np.nan, inplace=True)

    df['totalwgt_lb'] = df.birthwgt_lb + df.birthwgt_oz / 16.0  

This returns a dataframe with one row per record.

### 1.4: DataFrames

In [107]:
df = ReadFemPreg()
df

,caseid,pregordr,howpreg_n,howpreg_p,moscurrp,nowprgdk,pregend1,pregend2,nbrnaliv,multbrth,...,laborfor_i,religion_i,metro_i,basewgt,adj_mod_basewgt,finalwgt,secu_p,sest,cmintvw,totalwgt_lb
0,1,1,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,3410.389399,3869.349602,6448.271112,2,9,1231,8.8125
1,1,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,3410.389399,3869.349602,6448.271112,2,9,1231,7.8750
2,2,1,NaN,NaN,NaN,NaN,5.0,NaN,3.0,5.0,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,1231,9.1250
3,2,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,1231,7.0000
4,2,3,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,1231,6.1875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13588,12571,1,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,4670.540953,5795.692880,6269.200989,1,78,1227,6.1875
13589,12571,2,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,0,0,0,4670.540953,5795.692880,6269.200989,1,78,1227,NaN
13590,12571,3,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,0,0,0,4670.540953,5795.692880,6269.200989,1,78,1227,NaN
13591,12571,4,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,4670.540953,5795.692880,6269.200989,1,78,1227,7.5000


In [108]:
df.columns
# this returns a pandas index which is another pandas data structure. it behaves like a list

Index(['caseid', 'pregordr', 'howpreg_n', 'howpreg_p', 'moscurrp', 'nowprgdk',
       'pregend1', 'pregend2', 'nbrnaliv', 'multbrth',
       ...
       'laborfor_i', 'religion_i', 'metro_i', 'basewgt', 'adj_mod_basewgt',
       'finalwgt', 'secu_p', 'sest', 'cmintvw', 'totalwgt_lb'],
      dtype='object', length=244)

In [109]:
# pd index elements can be accessed with indices and slices
df.columns[0:3]

Index(['caseid', 'pregordr', 'howpreg_n'], dtype='object')

In [110]:
# df columns can be accessed with dot notation but only if the name is a "valid python
# identifier" - starts with a letter, does not contain spaces, etc.

df.caseid

# this returns a pd series - another list-like data structure but with more *features*

0            1
1            1
2            2
3            2
4            2
         ...  
13588    12571
13589    12571
13590    12571
13591    12571
13592    12571
Name: caseid, Length: 13593, dtype: int64

### 1.5: Variables
- caseid is the integer ID of the respondent.
- prglngth is the integer duration of the pregnancy in weeks.
- outcome is an integer code for the outcome of the pregnancy. The code 1 indicates a live birth.
- pregordr is a pregnancy serial number; for example, the code for a respondent’s first pregnancy is 1, for the second pregnancy is 2, and so on.
- birthord is a serial number for live births; the code for a respondent’s first child is 1, and so on. For outcomes other than live birth, this field is blank.
- birthwgt_lb and birthwgt_oz contain the pounds and ounces parts of the birth weight of the baby.
- agepreg is the mother’s age at the end of the pregnancy.
- finalwgt is the statistical weight associated with the respondent. It is a floating-point value that indicates the number of people in the U.S. population this respondent represents.

The codebook will tell you that many of the variables are `recodes` - they are not part of the raw data collected by the survey and are calculated after the fact. **Recodes are often based on logic that checks the consistency and accuracy of the data so it's a good idea to use them when available unless there is a good reason to process the raw data yourself.** 
>For example, prglngth for live births is equal to the raw variable wksgest (weeks of gestation) if it is available; otherwise it is estimated using mosgest * 4.33 (months of gestation times the average number of weeks in a month).


### 1.6: Transformation

In [111]:
def ReadFemPreg(dct_file='2002FemPreg.dct',
                dat_file='2002FemPreg.dat.gz'):
    dct = thinkstats2.ReadStataDct(dct_file)
    df = dct.ReadFixedWidth(dat_file, compression='gzip')
#     CleanFemPreg(df)
    return df

df = ReadFemPreg()

In [112]:
df.agepreg[:5]

0    3316.0
1    3925.0
2    1433.0
3    1783.0
4    1833.0
Name: agepreg, dtype: float64

In [113]:
df.birthwgt_lb[:5]

0    8.0
1    7.0
2    9.0
3    7.0
4    6.0
Name: birthwgt_lb, dtype: float64

Data cleaning involves checking for errors, handling special values, reformatting data, and doing calculations. this is what `CleanFemPreg(df)` is doing above. <br>
1. `df.agepreg` includes the Mother's age at the end of the pregnancy in **centiyears**. We'll convert this to normal people years.
2. weight is stored in two columns - one for pounds and one for ounces. We'll combine them to a single column in pounds and replace null values (97,98,99) with np.nan

**Watch out for special values encoded as numbers since they can mess stuff up!** Replacing them with NaN is a good idea since computations that include them will correctly return NaN <br>

When adding columns to a df you must use dictionary notation `df['newcol']` instead of dot notation - dot notation will just add an attribute to the df object.

In [114]:
def CleanFemPreg(df):
    df.agepreg /= 100.0

    na_vals = [97, 98, 99]
    df.birthwgt_lb.replace(na_vals, np.nan, inplace=True)
    df.birthwgt_oz.replace(na_vals, np.nan, inplace=True)

    df['totalwgt_lb'] = df.birthwgt_lb + df.birthwgt_oz / 16.0

In [115]:
CleanFemPreg(df)

In [116]:
df.agepreg[:5]

0    33.16
1    39.25
2    14.33
3    17.83
4    18.33
Name: agepreg, dtype: float64

In [117]:
df.totalwgt_lb[:5]

0    8.8125
1    7.8750
2    9.1250
3    7.0000
4    6.1875
Name: totalwgt_lb, dtype: float64

### 1.7: Validation
Validation is good. One way is to look at stats in the codebook and check that your data matches, for example this codebook provides summary tables with counts by outcome. You can compare this to your data by:

In [118]:
df.outcome.value_counts().sort_index()

1    9148
2    1862
3     120
4    1921
5     190
6     352
Name: outcome, dtype: int64

In [119]:
df.birthwgt_lb.value_counts().sort_index()

0.0        8
1.0       40
2.0       53
3.0       98
4.0      229
5.0      697
6.0     2223
7.0     3049
8.0     1889
9.0      623
10.0     132
11.0      26
12.0      10
13.0       3
14.0       3
15.0       1
51.0       1
Name: birthwgt_lb, dtype: int64

The 51 pound baby is most likely an error - let's get rid of it.

In [120]:
df.loc[df.birthwgt_lb > 20, 'birthwgt_lb'] = np.nan

the `df.loc` attribute provides methods to select rows and columns. the first expression is the indexer (which rows to select) and the second selects the column we want to assign a value to.
>The expression df.birthwgt_lb > 20 yields a Series of type bool, where True indicates that the condition is true. When a boolean Series is used as an index, it selects only the elements that satisfy the condition.

In [121]:
df.birthwgt_lb.value_counts().sort_index()

0.0        8
1.0       40
2.0       53
3.0       98
4.0      229
5.0      697
6.0     2223
7.0     3049
8.0     1889
9.0      623
10.0     132
11.0      26
12.0      10
13.0       3
14.0       3
15.0       1
Name: birthwgt_lb, dtype: int64

### 1.8: interpretation

In [122]:
def MakePregMap(df):
#     create a dictionary that maps from each case ID to a list of indices
    d = defaultdict(list)
#     enumerates the row number and case ID for each pregnancy
    for index, caseid in df.caseid.iteritems():
        d[caseid].append(index)
    return d

In [123]:
# choose a respondent
caseid = 10229
# create a dict with a key for each respondent ID that maps to a list of indices of their pregnancies
preg_map = nsfg.MakePregMap(df)
# get row numbers for this case ID
indices = preg_map[caseid]
# get values for the outcomes for this individual
df.outcome[indices].values

array([4, 4, 4, 4, 4, 4, 1])

### 1.9 Exercises

Select the `birthord` column, print the value counts, and compare to results published in the [codebook](http://www.icpsr.umich.edu/nsfg6/Controller?displayPage=labelDetails&fileCode=PREG&section=A&subSec=8016&srtLabel=611933)

In [124]:
df.birthord.value_counts()

1.0     4413
2.0     2874
3.0     1234
4.0      421
5.0      126
6.0       50
7.0       20
8.0        7
9.0        2
10.0       1
Name: birthord, dtype: int64

In [125]:
df.birthord.isnull().sum()
# this is cool - it counts the nulls in a column using .sum()

4445

Select the `prglngth` column, print the value counts, and compare to results published in the [codebook](http://www.icpsr.umich.edu/nsfg6/Controller?displayPage=labelDetails&fileCode=PREG&section=A&subSec=8016&srtLabel=611931)

In [126]:
df.prglngth.value_counts().sort_index()[:14].sum()

3522

Create a new column named totalwgt_kg that contains birth weight in kilograms. Compute its mean. Remember that when you create a new column, you have to use dictionary syntax, not dot notation.

In [130]:
df['totalwgt_kg'] = df.totalwgt_lb / 0.453592

In [136]:
df.totalwgt_kg.mean()

16.02874026149906

In [138]:
df.totalwgt_lb.mean()

7.270508352693882

nsfg.py also provides ReadFemResp, which reads the female respondents file and returns a DataFrame:

In [141]:
respdf = nsfg.ReadFemResp()

In [142]:
respdf.head()

,caseid,rscrinf,rdormres,rostscrn,rscreenhisp,rscreenrace,age_a,age_r,cmbirth,agescrn,...,pubassis_i,basewgt,adj_mod_basewgt,finalwgt,secu_r,sest,cmintvw,cmlstyr,screentime,intvlngth
0,2298,1,5,5,1,5.0,27,27,902,27,...,0,3247.916977,5123.759559,5556.717241,2,18,1234,1222,18:26:36,110.492667
1,5012,1,5,1,5,5.0,42,42,718,42,...,0,2335.279149,2846.799490,4744.191350,2,18,1233,1221,16:30:59,64.294000
2,11586,1,5,1,5,5.0,43,43,708,43,...,0,2335.279149,2846.799490,4744.191350,2,18,1234,1222,18:19:09,75.149167
3,6794,5,5,4,1,5.0,15,15,1042,15,...,0,3783.152221,5071.464231,5923.977368,2,18,1234,1222,15:54:43,28.642833
4,616,1,5,4,1,5.0,20,20,991,20,...,0,5341.329968,6437.335772,7229.128072,2,18,1233,1221,14:19:44,69.502667


In [152]:
respdf.age_r.value_counts().sort_index()

15    217
16    223
17    234
18    235
19    241
20    258
21    267
22    287
23    282
24    269
25    267
26    260
27    255
28    252
29    262
30    292
31    278
32    273
33    257
34    255
35    262
36    266
37    271
38    256
39    215
40    256
41    250
42    215
43    253
44    235
Name: age_r, dtype: int64

How old is the respondent with `caseid` 1?

In [154]:
respdf[respdf.caseid == 1]

,caseid,rscrinf,rdormres,rostscrn,rscreenhisp,rscreenrace,age_a,age_r,cmbirth,agescrn,...,pubassis_i,basewgt,adj_mod_basewgt,finalwgt,secu_r,sest,cmintvw,cmlstyr,screentime,intvlngth
1069,1,1,5,4,5,5.0,44,44,695,44,...,0,3410.389399,3869.349602,6448.271112,2,9,1231,1219,19:56:43,67.563833


What are the pregnancy lengths for the respondent with `caseid` 2298?

In [160]:
df[df.caseid == 2298].prglngth.values

array([40, 36, 30, 40])

What was the birthweight of the first baby born to the respondent with `caseid` 5012?

In [165]:
df[df.caseid == 5012].totalwgt_lb

5515    6.0
Name: totalwgt_lb, dtype: float64

### Book club intros
- What are you hoping to get out of this?
- How much python do you know, are you trying to learn?
- How much do you use pandas

### Book club discussion topics
- What are the pitfalls of using anecdotal evidence?
- Hypothesis testing - where apparent effects are observed, might they have happened by chance?
- cross-sectional vs longitudinal studies
- vocabulary - sample, population, representative, oversampled
- Stata (statistical software system)
- defaultdict()